In [29]:
import pandas as pd
import numpy as np
import seaborn as sns

import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder

from models.mlp import BlackBoxModel

pd.set_option('display.max_columns', None)

%reload_ext autoreload
%autoreload 2

In [30]:
df_ = pd.read_csv('hotel_bookings.csv')
df = df_.copy()
target_name = 'is_canceled'
target = df[target_name]

In [31]:
# Initialize a label encoder
label_encoder = LabelEncoder()
label_mappings = {}


# Convert categorical columns to numerical representations using label encoding
for column in df.columns:
    if df[column].dtype == 'object':
        # Handle missing values by filling with a placeholder and then encoding
        df[column] = df[column].fillna('Unknown')
        df[column] = label_encoder.fit_transform(df[column])
        label_mappings[column] = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))


# For columns with NaN values that are numerical, we will impute them with the median of the column
for column in df.columns:
    if df[column].isna().any():
        median_val = df[column].median()
        df[column].fillna(median_val, inplace=True)

# Display the first few rows of the transformed dataframe
df.head()


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,1,0,342,2015,5,27,1,0,0,2,0.0,0,0,135,3,1,0,0,0,2,2,3,0,14.0,179.0,0,2,0.0,0,0,1,121
1,1,0,737,2015,5,27,1,0,0,2,0.0,0,0,135,3,1,0,0,0,2,2,4,0,14.0,179.0,0,2,0.0,0,0,1,121
2,1,0,7,2015,5,27,1,0,1,1,0.0,0,0,59,3,1,0,0,0,0,2,0,0,14.0,179.0,0,2,75.0,0,0,1,122
3,1,0,13,2015,5,27,1,0,1,1,0.0,0,0,59,2,0,0,0,0,0,0,0,0,304.0,179.0,0,2,75.0,0,0,1,122
4,1,0,14,2015,5,27,1,0,2,2,0.0,0,0,59,6,3,0,0,0,0,0,0,0,240.0,179.0,0,2,98.0,0,1,1,123


In [32]:
features = [
    # 'hotel', 
    'lead_time', 
    # 'arrival_date_year', 
    'arrival_date_month',
    # 'arrival_date_week_number', 
    # 'arrival_date_day_of_month',
    'stays_in_weekend_nights', 
    'stays_in_week_nights', 
    # 'adults', 
    # 'children',
    # 'babies', 
    # 'meal', 
    'country', 
    'market_segment', 
    # 'distribution_channel',
    # 'is_repeated_guest', 
    'previous_cancellations',
    # 'previous_bookings_not_canceled', 
    'reserved_room_type',
    'assigned_room_type', 
    'booking_changes', 
    'deposit_type', 
    'agent',
    # 'company', 
    'days_in_waiting_list', 
    'customer_type', 
    'adr',
    'required_car_parking_spaces', 
    'total_of_special_requests'
]

df_X = df[features].copy()
df_y = target

In [33]:
seed = 42

np.random.seed(seed)  # for reproducibility


# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=seed)

std = X_train.std()
mean = X_train.mean()

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1)

# Initialize the model, loss function, and optimizer
model = BlackBoxModel(input_dim=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluate on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)

    # Convert outputs to binary using 0.5 as threshold
    y_pred_tensor = (test_outputs > 0.5).float()
    correct_predictions = (y_pred_tensor == y_test_tensor).float().sum()
    accuracy = correct_predictions / y_test_tensor.shape[0]

accuracy.item()

0.8220118880271912

In [109]:
sample_num = 25

indice = (X_test.sample(sample_num)).index

In [110]:
X_prime = X_test.loc[indice].values
X = X_prime.copy()

y_prime = model(torch.FloatTensor(X_prime))
y = model(torch.FloatTensor(X))

y_true = y_test.loc[indice]

In [137]:
from explainers.dce import DistributionalCounterfactualExplainer

explainer = DistributionalCounterfactualExplainer(model=model, X=X, lr=1e-1, epsilon=0.5, lambda_val=10)

In [138]:
explainer.optimize(max_iter=100)

Iter 1: Q = 4.118701934814453, term1 = 0.3365967571735382, term2 = 3.782104969024658
Iter 2: Q = 3.976466178894043, term1 = 0.2594038248062134, term2 = 3.717062473297119
Iter 3: Q = 3.8905720710754395, term1 = 0.19894561171531677, term2 = 3.69162654876709
Iter 4: Q = 3.80959153175354, term1 = 0.15364496409893036, term2 = 3.6559464931488037
Iter 5: Q = 3.716304302215576, term1 = 0.12043478339910507, term2 = 3.595869541168213
Iter 6: Q = 3.650381326675415, term1 = 0.09647698700428009, term2 = 3.5539042949676514
Iter 7: Q = 3.586362838745117, term1 = 0.08042925596237183, term2 = 3.5059335231781006
Iter 8: Q = 3.5358474254608154, term1 = 0.0682886391878128, term2 = 3.4675588607788086
Iter 9: Q = 3.4838359355926514, term1 = 0.06135579198598862, term2 = 3.4224801063537598
Iter 10: Q = 3.4319119453430176, term1 = 0.05447251722216606, term2 = 3.377439498901367
Iter 11: Q = 3.3973419666290283, term1 = 0.050361327826976776, term2 = 3.346980571746826
Iter 12: Q = 3.378502130508423, term1 = 0.0477

In [123]:
factual_X = df[df_X.columns].loc[indice].copy()
counterfactual_X = pd.DataFrame(explainer.X.detach().numpy() * std[df_X.columns].values + mean[df_X.columns].values, columns=df_X.columns)
# counterfactual_X = pd.DataFrame(explainer.X.detach().numpy(), columns=df_X.columns)

factual_y = pd.DataFrame(y.detach().numpy(),columns=[target_name], index=factual_X.index)
counterfactual_y = pd.DataFrame(explainer.y.detach().numpy(),columns=[target_name], index=factual_X.index)

In [124]:
factual_y.mean()

is_canceled    0.354351
dtype: float32

In [125]:
counterfactual_y.mean()

is_canceled    0.400381
dtype: float32

In [126]:
check_column = 'lead_time'
pd.DataFrame({
    'factual': factual_X[check_column].values, 
    'counterfactual': counterfactual_X[check_column].values
    })

,factual,counterfactual
0,40,-21.081272
1,405,311.588110
2,31,-52.224129
3,11,193.796443
4,4,-300.802265
5,195,-2.832608
6,162,219.310616
7,178,288.306611
8,15,203.111323
9,104,238.595328


In [127]:
factual_X[check_column].mean()

101.92

In [128]:
counterfactual_X[check_column].mean()

97.08619458605484